In [1]:
import torch
import torch.nn as nn
import csv
import json
import math
from torch.autograd import Variable
from pytorch_pretrained_bert.tokenization import BertTokenizer
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool
from transformer_nb import *

In [2]:
task_name = "giga"

if task_name == "giga":
    doc_name = "/home/george/Projects/speechlab/pointer-generator/data/Giga/input.txt"
    summ_name = "/home/george/Projects/speechlab/pointer-generator/data/Giga/task1_ref0.txt"
else:
    doc_name = "/home/george/Projects/speechlab/pointer-generator/data2/val.txt.src"
    summ_name = "/home/george/Projects/speechlab/pointer-generator/data2/val.txt.tgt.tagged"
    
val_size = 1951 if task_name == "giga" else 13368

continue_from = "trained/Model2"
eval_dir = "evaluation-{}/".format(task_name)
data_seq_name = eval_dir+'tmp.json'
vocab_name = 'data-giga-light/vocab.json'

num_threads = 4
batch_size = 32 if task_name == "giga" else 20
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
# vocab = tokenizer.vocab
vocab = json.load(open(vocab_name, 'r'))
VOC_SIZE = len(vocab)

In [3]:
!mkdir -p {eval_dir+'gold/'}
!mkdir -p {eval_dir+'mine/'}

In [4]:
INPUT_MAX = 50 if task_name == "giga" else 400
OUTPUT_MAX = 20 if task_name == "giga" else 100
UNK = "[UNK]"
BOS = "[CLS]"
EOS = "[SEP]"
PAD = "[PAD]"

In [5]:
# if not using BERT tokenizer
# import wordninja
# class Tokenizer:
#     def __init__(self):
#         self.vocab = vocab
#         self.vocab_inv = {a:b for b, a in vocab.items()}
#     def convert_ids_to_tokens(self, ids):
#         return [self.vocab_inv[i] for i in ids]
#     def convert_tokens_to_ids(self, tokens):
#         return [(self.vocab[t] if t in self.vocab else self.vocab[UNK]) for t in tokens]
#     def tokenize(self, s):
#         return wordninja.split(s)
# tokenizer = Tokenizer()

In [6]:
if task_name == 'giga':
    token_mappings = {'UNK':UNK, '-lrb-':'(', '-rrb-':')'}
else:
    token_mappings = {'<unk>':UNK, '<t>':'', '</t>':''}

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda


In [8]:
def getlines(name):
    total = !wc -l {name}
    return int(total[0].split()[0])

In [9]:
def swap(s):
    for t, t_p in token_mappings.items():
        s = s.replace(t, t_p)
    if s == "":
        s = UNK
    return s

In [10]:
summaries = []
articles = []

with open(doc_name, newline='', encoding='utf-8') as f:
    total = getlines(doc_name)
    for i,line in tqdm(enumerate(f), total=total):
        line = line.strip()
        articles.append(swap(line))
with open(summ_name, newline='', encoding='utf-8') as f:
    total = getlines(summ_name)
    for i,line in tqdm(enumerate(f), total=total):
        line = line.strip()
        summaries.append(swap(line))
        
print(len(summaries))


1951


In [11]:
summaries = summaries[:val_size]
articles = articles[:val_size]
print(len(summaries))

1951


In [12]:
def get_dig(n):
    return 1 if n < 10 else (get_dig(n/10) + 1)

digits = get_dig(val_size)
print(digits)

def make_gold(summ):
    for i,s in tqdm(enumerate(summ), total=len(summ)):
        filename = (eval_dir+'gold/'+task_name+'.A.{:0'+str(digits)+'d}.txt').format(i+1)
        with open(filename, 'w') as f:
            f.write(s)
# make_gold(summaries)

4


In [13]:
def task(s):
    for t, t_p in token_mappings.items():
        s = s.replace(t, t_p)
    ts = tokenizer.tokenize(s)
    return [BOS] + ts + [EOS]

In [14]:
with Pool(num_threads) as p:
    chunksize = 100
    summaries = list(tqdm(p.imap(task, summaries, chunksize=chunksize), total=len(summaries)))
    articles = list(tqdm(p.imap(task, articles, chunksize=chunksize), total=len(articles)))

In [15]:
index = 73
print("[summary]", summaries[index])
print("[documen]", articles[index])

[summary] ['[CLS]', 'russia', 'warns', 'of', 'colossal', 'impact', 'if', 'nato', 'takes', 'in', 'ukraine', 'georgia', '[', '[', 'un', '##k', ']', ']', '[', '[', 'un', '##k', ']', ']', 'quote', '[SEP]']
[documen] ['[CLS]', 'russia', 'warned', 'wednesday', 'against', 'nato', 'taking', 'in', 'the', 'ex', '-', 'soviet', 'republics', 'of', 'ukraine', 'and', 'georgia', ',', 'saying', 'such', 'a', 'colossal', 'geo', '##pol', '##itical', 'shift', 'would', 'threaten', 'relations', '.', '[SEP]']


In [16]:
seqdata = {'summary':[], 'document':[]}
num_summ = len(summaries)
summ_seqs = [tokenizer.convert_tokens_to_ids(s) for s in tqdm(summaries)]
docu_seqs = [tokenizer.convert_tokens_to_ids(s) for s in tqdm(articles)]

for i in range(num_summ):    
    seqdata['summary'].append(summ_seqs[i])
    seqdata['document'].append(docu_seqs[i])

json.dump(seqdata, open(data_seq_name, 'w'))

In [17]:
from dataset import make_data_generator
# batch_size = 20
ev_set, ev_generator = make_data_generator(\
data_seq_name, INPUT_MAX, OUTPUT_MAX, vocab[PAD], batch_size, cutoff=None, shuffle=False, num_workers=4)

loading json
load json done.


In [18]:
def data_gen_val():
    for src,tgt in ev_generator:
        src = Variable(src, requires_grad=False).to(device)
        tgt = Variable(tgt, requires_grad=False).to(device)
        yield Batch(src, tgt, vocab[PAD])

In [19]:
if str(device) == 'cpu':
    saved_model = torch.load(continue_from, map_location=lambda storage, location: storage)
else:
    saved_model = torch.load(continue_from)

model = make_model(VOC_SIZE, VOC_SIZE, N=4, d_model=256, d_ff=1024, h=8, dropout=0.1)
model.load_state_dict(saved_model['model'])
model.eval()
if str(device) != 'cpu':
    model.cuda()

In [20]:
del saved_model
torch.cuda.empty_cache()

In [21]:
def greedy_decode_batch(model, src, src_mask, max_len, start_symbol):
    batch_size = src.shape[0]
    
    memory = model.encode(src, src_mask)
    ys = torch.ones(batch_size, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len-1):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        #print(out.shape) 128,1,256
        probs = model.generator(out[:, -1, :])
        
        #print(probs.shape) 128,30522
        next_words = torch.argmax(probs, dim=1, keepdim=True)
        
        #print(next_words.shape)        
        #print(ys.shape) both 128,1
        
        ys = torch.cat((ys, next_words), dim=1)
    return ys

In [22]:
def readable(sent):
    try:
        end = sent.index(EOS)
    except ValueError:
        end = len(sent)
    sent = [tok for tok in sent[:end] if tok not in [BOS, EOS, PAD]] # remove special tokens
    sent = " ".join(sent)
#     for t, t_p in token_mappings.items():
#         if t_p != '':
#             sent = sent.replace(t_p, t)
    return sent
def readable_old(sent):
    try:
        end = sent.index(EOS)
    except ValueError:
        end = len(sent)
    sent = " ".join(sent[:end])
    sent = sent.replace(BOS, '')
    sent = sent.replace(EOS, '')
    sent = sent.replace(UNK, '-UNK-')
    sent = sent.replace(PAD, '')
    return sent

In [23]:
examples = min(100000, ev_set.size)
print(examples)

total = int(math.ceil(examples / batch_size))


for i,batch in tqdm(enumerate(data_gen_val()), total= total):
    srcs = batch.src
    src_masks = batch.src_mask
    
    trgs = batch.trg
    trg_masks = batch.trg_mask
        
    bs = srcs.shape[0]
    
    outs = greedy_decode_batch(model, srcs, None, max_len=OUTPUT_MAX, start_symbol=vocab[BOS])
    
    for j, (out_tensor, tgt_tensor) in enumerate(zip(outs, trgs)):        
        tokens = tokenizer.convert_ids_to_tokens(out_tensor.cpu().numpy())        
        line = readable(tokens)
        #print(line)
        numeric = i*batch_size + j + 1
        filename = (eval_dir+'mine/'+task_name+'.{:0'+str(digits)+'d}.txt').format(numeric)
        
        with open(filename, 'w') as mine:
            mine.write(line)
            
    if i == total:
        break

1951


In [24]:

from pyrouge import Rouge155

r = Rouge155()
r.system_dir = eval_dir+'mine/'
r.model_dir = eval_dir+'gold/'
r.system_filename_pattern = task_name+'.(\d+).txt'
r.model_filename_pattern = task_name+'.[A-Z].#ID#.txt'

output = r.convert_and_evaluate()

2019-07-26 15:21:43,159 [MainThread  ] [INFO ]  Writing summaries.
2019-07-26 15:21:43,161 [MainThread  ] [INFO ]  Processing summaries. Saving system files to /tmp/tmp0ujah5w4/system and model files to /tmp/tmp0ujah5w4/model.
2019-07-26 15:21:43,162 [MainThread  ] [INFO ]  Processing files in evaluation-giga/mine/.
2019-07-26 15:21:43,164 [MainThread  ] [INFO ]  Processing giga.0032.txt.
2019-07-26 15:21:43,166 [MainThread  ] [INFO ]  Processing giga.1446.txt.
2019-07-26 15:21:43,172 [MainThread  ] [INFO ]  Processing giga.0030.txt.
2019-07-26 15:21:43,174 [MainThread  ] [INFO ]  Processing giga.1471.txt.
2019-07-26 15:21:43,177 [MainThread  ] [INFO ]  Processing giga.1356.txt.
2019-07-26 15:21:43,177 [MainThread  ] [INFO ]  Processing giga.0639.txt.
2019-07-26 15:21:43,179 [MainThread  ] [INFO ]  Processing giga.0796.txt.
2019-07-26 15:21:43,180 [MainThread  ] [INFO ]  Processing giga.1077.txt.
2019-07-26 15:21:43,180 [MainThread  ] [INFO ]  Processing giga.0182.txt.
2019-07-26 15:21

2019-07-26 15:21:43,252 [MainThread  ] [INFO ]  Processing giga.0866.txt.
2019-07-26 15:21:43,253 [MainThread  ] [INFO ]  Processing giga.1278.txt.
2019-07-26 15:21:43,253 [MainThread  ] [INFO ]  Processing giga.0525.txt.
2019-07-26 15:21:43,253 [MainThread  ] [INFO ]  Processing giga.0024.txt.
2019-07-26 15:21:43,254 [MainThread  ] [INFO ]  Processing giga.1553.txt.
2019-07-26 15:21:43,254 [MainThread  ] [INFO ]  Processing giga.1947.txt.
2019-07-26 15:21:43,255 [MainThread  ] [INFO ]  Processing giga.0193.txt.
2019-07-26 15:21:43,255 [MainThread  ] [INFO ]  Processing giga.0989.txt.
2019-07-26 15:21:43,256 [MainThread  ] [INFO ]  Processing giga.0440.txt.
2019-07-26 15:21:43,256 [MainThread  ] [INFO ]  Processing giga.0816.txt.
2019-07-26 15:21:43,257 [MainThread  ] [INFO ]  Processing giga.1327.txt.
2019-07-26 15:21:43,257 [MainThread  ] [INFO ]  Processing giga.1559.txt.
2019-07-26 15:21:43,258 [MainThread  ] [INFO ]  Processing giga.1585.txt.
2019-07-26 15:21:43,258 [MainThread  ]

2019-07-26 15:21:43,309 [MainThread  ] [INFO ]  Processing giga.0752.txt.
2019-07-26 15:21:43,310 [MainThread  ] [INFO ]  Processing giga.0631.txt.
2019-07-26 15:21:43,310 [MainThread  ] [INFO ]  Processing giga.1182.txt.
2019-07-26 15:21:43,311 [MainThread  ] [INFO ]  Processing giga.0611.txt.
2019-07-26 15:21:43,311 [MainThread  ] [INFO ]  Processing giga.1367.txt.
2019-07-26 15:21:43,311 [MainThread  ] [INFO ]  Processing giga.0735.txt.
2019-07-26 15:21:43,312 [MainThread  ] [INFO ]  Processing giga.0862.txt.
2019-07-26 15:21:43,313 [MainThread  ] [INFO ]  Processing giga.0785.txt.
2019-07-26 15:21:43,313 [MainThread  ] [INFO ]  Processing giga.0728.txt.
2019-07-26 15:21:43,314 [MainThread  ] [INFO ]  Processing giga.0491.txt.
2019-07-26 15:21:43,315 [MainThread  ] [INFO ]  Processing giga.1711.txt.
2019-07-26 15:21:43,315 [MainThread  ] [INFO ]  Processing giga.0195.txt.
2019-07-26 15:21:43,315 [MainThread  ] [INFO ]  Processing giga.0998.txt.
2019-07-26 15:21:43,316 [MainThread  ]

2019-07-26 15:21:43,365 [MainThread  ] [INFO ]  Processing giga.1832.txt.
2019-07-26 15:21:43,365 [MainThread  ] [INFO ]  Processing giga.0389.txt.
2019-07-26 15:21:43,366 [MainThread  ] [INFO ]  Processing giga.0574.txt.
2019-07-26 15:21:43,367 [MainThread  ] [INFO ]  Processing giga.1666.txt.
2019-07-26 15:21:43,367 [MainThread  ] [INFO ]  Processing giga.1934.txt.
2019-07-26 15:21:43,368 [MainThread  ] [INFO ]  Processing giga.0331.txt.
2019-07-26 15:21:43,369 [MainThread  ] [INFO ]  Processing giga.0488.txt.
2019-07-26 15:21:43,369 [MainThread  ] [INFO ]  Processing giga.0563.txt.
2019-07-26 15:21:43,370 [MainThread  ] [INFO ]  Processing giga.1505.txt.
2019-07-26 15:21:43,370 [MainThread  ] [INFO ]  Processing giga.0029.txt.
2019-07-26 15:21:43,371 [MainThread  ] [INFO ]  Processing giga.1792.txt.
2019-07-26 15:21:43,371 [MainThread  ] [INFO ]  Processing giga.0505.txt.
2019-07-26 15:21:43,372 [MainThread  ] [INFO ]  Processing giga.1950.txt.
2019-07-26 15:21:43,372 [MainThread  ]

2019-07-26 15:21:43,471 [MainThread  ] [INFO ]  Processing giga.1820.txt.
2019-07-26 15:21:43,472 [MainThread  ] [INFO ]  Processing giga.0531.txt.
2019-07-26 15:21:43,472 [MainThread  ] [INFO ]  Processing giga.0874.txt.
2019-07-26 15:21:43,473 [MainThread  ] [INFO ]  Processing giga.1932.txt.
2019-07-26 15:21:43,474 [MainThread  ] [INFO ]  Processing giga.0351.txt.
2019-07-26 15:21:43,474 [MainThread  ] [INFO ]  Processing giga.1919.txt.
2019-07-26 15:21:43,475 [MainThread  ] [INFO ]  Processing giga.0672.txt.
2019-07-26 15:21:43,475 [MainThread  ] [INFO ]  Processing giga.0666.txt.
2019-07-26 15:21:43,476 [MainThread  ] [INFO ]  Processing giga.0792.txt.
2019-07-26 15:21:43,476 [MainThread  ] [INFO ]  Processing giga.0875.txt.
2019-07-26 15:21:43,477 [MainThread  ] [INFO ]  Processing giga.0115.txt.
2019-07-26 15:21:43,480 [MainThread  ] [INFO ]  Processing giga.0179.txt.
2019-07-26 15:21:43,487 [MainThread  ] [INFO ]  Processing giga.0021.txt.
2019-07-26 15:21:43,487 [MainThread  ]

...

2019-07-26 15:21:45,063 [MainThread  ] [INFO ]  Processing giga.A.0968.txt.
2019-07-26 15:21:45,064 [MainThread  ] [INFO ]  Processing giga.A.1301.txt.
2019-07-26 15:21:45,065 [MainThread  ] [INFO ]  Processing giga.A.0241.txt.
2019-07-26 15:21:45,065 [MainThread  ] [INFO ]  Processing giga.A.0732.txt.
2019-07-26 15:21:45,066 [MainThread  ] [INFO ]  Processing giga.A.1266.txt.
2019-07-26 15:21:45,067 [MainThread  ] [INFO ]  Processing giga.A.1917.txt.
2019-07-26 15:21:45,068 [MainThread  ] [INFO ]  Processing giga.A.1234.txt.
2019-07-26 15:21:45,068 [MainThread  ] [INFO ]  Processing giga.A.0287.txt.
2019-07-26 15:21:45,069 [MainThread  ] [INFO ]  Processing giga.A.0400.txt.
2019-07-26 15:21:45,070 [MainThread  ] [INFO ]  Processing giga.A.1243.txt.
2019-07-26 15:21:45,070 [MainThread  ] [INFO ]  Processing giga.A.0961.txt.
2019-07-26 15:21:45,071 [MainThread  ] [INFO ]  Processing giga.A.0127.txt.
2019-07-26 15:21:45,072 [MainThread  ] [INFO ]  Processing giga.A.0604.txt.
2019-07-26 1

2019-07-26 15:21:45,133 [MainThread  ] [INFO ]  Processing giga.A.1794.txt.
2019-07-26 15:21:45,134 [MainThread  ] [INFO ]  Processing giga.A.1628.txt.
2019-07-26 15:21:45,134 [MainThread  ] [INFO ]  Processing giga.A.0715.txt.
2019-07-26 15:21:45,135 [MainThread  ] [INFO ]  Processing giga.A.1206.txt.
2019-07-26 15:21:45,135 [MainThread  ] [INFO ]  Processing giga.A.0522.txt.
2019-07-26 15:21:45,136 [MainThread  ] [INFO ]  Processing giga.A.0145.txt.
2019-07-26 15:21:45,136 [MainThread  ] [INFO ]  Processing giga.A.1421.txt.
2019-07-26 15:21:45,137 [MainThread  ] [INFO ]  Processing giga.A.0724.txt.
2019-07-26 15:21:45,137 [MainThread  ] [INFO ]  Processing giga.A.0210.txt.
2019-07-26 15:21:45,137 [MainThread  ] [INFO ]  Processing giga.A.1892.txt.
2019-07-26 15:21:45,138 [MainThread  ] [INFO ]  Processing giga.A.1688.txt.
2019-07-26 15:21:45,138 [MainThread  ] [INFO ]  Processing giga.A.1219.txt.
2019-07-26 15:21:45,139 [MainThread  ] [INFO ]  Processing giga.A.0460.txt.
2019-07-26 1

2019-07-26 15:21:45,180 [MainThread  ] [INFO ]  Processing giga.A.1227.txt.
2019-07-26 15:21:45,180 [MainThread  ] [INFO ]  Processing giga.A.0162.txt.
2019-07-26 15:21:45,181 [MainThread  ] [INFO ]  Processing giga.A.0718.txt.
2019-07-26 15:21:45,181 [MainThread  ] [INFO ]  Processing giga.A.1051.txt.
2019-07-26 15:21:45,181 [MainThread  ] [INFO ]  Processing giga.A.0281.txt.
2019-07-26 15:21:45,182 [MainThread  ] [INFO ]  Processing giga.A.0662.txt.
2019-07-26 15:21:45,182 [MainThread  ] [INFO ]  Processing giga.A.0304.txt.
2019-07-26 15:21:45,183 [MainThread  ] [INFO ]  Processing giga.A.0369.txt.
2019-07-26 15:21:45,183 [MainThread  ] [INFO ]  Processing giga.A.0817.txt.
2019-07-26 15:21:45,184 [MainThread  ] [INFO ]  Processing giga.A.0144.txt.
2019-07-26 15:21:45,184 [MainThread  ] [INFO ]  Processing giga.A.1742.txt.
2019-07-26 15:21:45,185 [MainThread  ] [INFO ]  Processing giga.A.0084.txt.
2019-07-26 15:21:45,185 [MainThread  ] [INFO ]  Processing giga.A.0699.txt.
2019-07-26 1

2019-07-26 15:21:45,235 [MainThread  ] [INFO ]  Processing giga.A.0758.txt.
2019-07-26 15:21:45,235 [MainThread  ] [INFO ]  Processing giga.A.0558.txt.
2019-07-26 15:21:45,236 [MainThread  ] [INFO ]  Processing giga.A.1438.txt.
2019-07-26 15:21:45,236 [MainThread  ] [INFO ]  Processing giga.A.1792.txt.
2019-07-26 15:21:45,237 [MainThread  ] [INFO ]  Processing giga.A.1015.txt.
2019-07-26 15:21:45,237 [MainThread  ] [INFO ]  Processing giga.A.0554.txt.
2019-07-26 15:21:45,237 [MainThread  ] [INFO ]  Processing giga.A.0441.txt.
2019-07-26 15:21:45,238 [MainThread  ] [INFO ]  Processing giga.A.1639.txt.
2019-07-26 15:21:45,238 [MainThread  ] [INFO ]  Processing giga.A.1520.txt.
2019-07-26 15:21:45,244 [MainThread  ] [INFO ]  Processing giga.A.1270.txt.
2019-07-26 15:21:45,244 [MainThread  ] [INFO ]  Processing giga.A.0769.txt.
2019-07-26 15:21:45,245 [MainThread  ] [INFO ]  Processing giga.A.0172.txt.
2019-07-26 15:21:45,245 [MainThread  ] [INFO ]  Processing giga.A.0059.txt.
2019-07-26 1

2019-07-26 15:21:45,285 [MainThread  ] [INFO ]  Processing giga.A.0748.txt.
2019-07-26 15:21:45,286 [MainThread  ] [INFO ]  Processing giga.A.1608.txt.
2019-07-26 15:21:45,286 [MainThread  ] [INFO ]  Processing giga.A.0991.txt.
2019-07-26 15:21:45,287 [MainThread  ] [INFO ]  Processing giga.A.1673.txt.
2019-07-26 15:21:45,287 [MainThread  ] [INFO ]  Processing giga.A.0918.txt.
2019-07-26 15:21:45,287 [MainThread  ] [INFO ]  Processing giga.A.0236.txt.
2019-07-26 15:21:45,288 [MainThread  ] [INFO ]  Processing giga.A.0673.txt.
2019-07-26 15:21:45,288 [MainThread  ] [INFO ]  Processing giga.A.0101.txt.
2019-07-26 15:21:45,289 [MainThread  ] [INFO ]  Processing giga.A.0220.txt.
2019-07-26 15:21:45,289 [MainThread  ] [INFO ]  Processing giga.A.0019.txt.
2019-07-26 15:21:45,289 [MainThread  ] [INFO ]  Processing giga.A.0361.txt.
2019-07-26 15:21:45,290 [MainThread  ] [INFO ]  Processing giga.A.0706.txt.
2019-07-26 15:21:45,290 [MainThread  ] [INFO ]  Processing giga.A.1109.txt.
2019-07-26 1

In [25]:
print(output)

---------------------------------------------
1 ROUGE-1 Average_R: 0.29201 (95%-conf.int. 0.28130 - 0.30304)
1 ROUGE-1 Average_P: 0.31509 (95%-conf.int. 0.30390 - 0.32678)
1 ROUGE-1 Average_F: 0.29139 (95%-conf.int. 0.28137 - 0.30220)
---------------------------------------------
1 ROUGE-2 Average_R: 0.12457 (95%-conf.int. 0.11570 - 0.13299)
1 ROUGE-2 Average_P: 0.13235 (95%-conf.int. 0.12322 - 0.14161)
1 ROUGE-2 Average_F: 0.12318 (95%-conf.int. 0.11444 - 0.13141)
---------------------------------------------
1 ROUGE-3 Average_R: 0.05991 (95%-conf.int. 0.05280 - 0.06674)
1 ROUGE-3 Average_P: 0.06349 (95%-conf.int. 0.05562 - 0.07074)
1 ROUGE-3 Average_F: 0.05918 (95%-conf.int. 0.05191 - 0.06588)
---------------------------------------------
1 ROUGE-4 Average_R: 0.03224 (95%-conf.int. 0.02636 - 0.03810)
1 ROUGE-4 Average_P: 0.03391 (95%-conf.int. 0.02773 - 0.04030)
1 ROUGE-4 Average_F: 0.03189 (95%-conf.int. 0.02601 - 0.03780)
---------------------------------------------
1 ROUGE-L Aver